# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [69]:
import pandas as pd
from data_cleaning import clean_all, clean_column_names, clean_customer_data, format_data_types, create_active_df, remove_duplicates

# Load the dataset
url = 'https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv'
df = pd.read_csv(url)
df.head()

,Unnamed: 0,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,...,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size,Vehicle Type
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2/18/11,Employed,M,...,0.0,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228.525238,No,Basic,College,1/18/11,Unemployed,F,...,0.0,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2/10/11,Employed,M,...,0.0,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,1/11/11,Employed,M,...,0.0,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,1/17/11,Medical Leave,F,...,NaN,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,NaN


In [72]:
# Calling the functions and cleaning the data frames
import importlib
import data_cleaning
importlib.reload(data_cleaning)

df = clean_all(df)
df.head()

/Users/eppuhenriksson/Documents/IronHack/Week 2/labs/lab-dw-data-aggregation-and-filtering/data_cleaning.py:120: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  # Try to convert all columns to numeric where possible


,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
0,0,DK49336,Arizona,4809,No,Basic,College,2/18/11,Employed,M,...,0,9,Corporate Auto,Corporate L3,Offer3,Agent,292,Four-Door Car,Medsize,NaN
1,1,KX64629,California,2228,No,Basic,College,1/18/11,Unemployed,F,...,0,1,Personal Auto,Personal L3,Offer4,Call Center,744,Four-Door Car,Medsize,NaN
2,2,LZ68649,Washington,14947,No,Basic,Bachelor,2/10/11,Employed,M,...,0,2,Personal Auto,Personal L3,Offer3,Call Center,480,SUV,Medsize,A
3,3,XL78013,Oregon,22332,Yes,Extended,College,1/11/11,Employed,M,...,0,2,Corporate Auto,Corporate L3,Offer2,Branch,484,Four-Door Car,Medsize,A
5,5,HO81102,Nan,4745,NaN,Basic,High School or Below,2/14/11,Employed,M,...,0,7,Personal Auto,Personal L3,Offer1,Agent,292,Four-Door Car,Medsize,A


1. Create a new DataFrame that only includes customers who:
   - have a **low total_claim_amount** (e.g., below $1,000),
   - have a response "Yes" to the last marketing campaign.

In [63]:
df1 = df[(df.total_claim_amount < 1000) & (df.response == "Yes")]
df1.head()

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type
3,3,XL78013,Oregon,22332,Yes,Extended,College,1/11/11,Employed,M,...,0,2,Corporate Auto,Corporate L3,Offer2,Branch,484,Four-Door Car,Medsize,A
8,8,FM55990,California,5989,Yes,Premium,College,1/19/11,Employed,M,...,0,1,Personal Auto,Personal L1,Offer2,Branch,739,Luxury,Medsize,NaN
15,15,CW49887,California,4626,Yes,Basic,Master,1/16/11,Employed,F,...,0,1,Special Auto,Special L1,Offer2,Branch,547,SUV,Medsize,NaN
19,19,NJ54277,California,3746,Yes,Extended,College,2/26/11,Employed,F,...,1,1,Personal Auto,Personal L2,Offer2,Call Center,19,Two-Door Car,Large,A
27,27,MQ68407,Oregon,4376,Yes,Premium,Bachelor,2/28/11,Employed,F,...,0,1,Personal Auto,Personal L3,Offer2,Agent,60,Four-Door Car,Medsize,NaN


2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

In [51]:
df2 = df[df.response == "Yes"]
table2_df = df2.groupby(['gender','policy_type']).agg({"monthly_premium_auto" : "mean", 
                                                    "total_claim_amount" : "mean" })
table2_df["ratio"] = table2_df['total_claim_amount']/table2_df['monthly_premium_auto']
table2_df

monthly_premium_auto  total_claim_amount     ratio
gender policy_type                                                       
F      Corporate Auto             94.301775          433.738499  4.599473
       Personal Auto              98.998148          452.965929  4.575499
       Special Auto               92.314286          453.280164  4.910184
M      Corporate Auto             92.188312          408.582459  4.432042
       Personal Auto              91.085821          457.010178  5.017358
       Special Auto               86.343750          429.527942  4.974627

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

In [52]:
# Count customers per state
state_counts = df.groupby('state')['customer'].count()

# Filter to include only states with more than 500 customers
states_over_500 = state_counts[state_counts > 500]

# Display the result
print(states_over_500)

state
Arizona       1937
California    3552
Nevada         993
Oregon        2909
Washington     888
Name: customer, dtype: int64


4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

In [53]:
table4_df = df1.groupby(['education', 'gender']).agg({"customer_lifetime_value" : ["max", "min", "mean"] })
table4_df

customer_lifetime_value                           
                                                max          min          mean
education            gender                                                   
Bachelor             F                 33473.349460  2248.449633   7141.659551
                     M                 24127.504020  2300.691547   8647.850455
College              F                 25464.820590  2004.350666   6407.510164
                     M                 22332.439460  2393.915369   7542.531090
Doctor               F                  6265.343299  2395.570000   4501.957277
                     M                 12731.951610  2491.317024   6544.019839
High School or Below F                 41787.903430  2397.036098  10679.830995
                     M                 12298.686300  2227.072755   6731.080236
Master               F                 16301.967600  2574.020376   6384.506048
                     M                 14435.673650  2551.226692   7738.723086

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

In [77]:
# Convert to datetime if not already
df['effective_to_date'] = pd.to_datetime(df['effective_to_date'])

# Extract month name
df['month'] = df['effective_to_date'].dt.month_name()
df

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809,No,Basic,College,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292,Four-Door Car,Medsize,NaN,February
1,1,KX64629,California,2228,No,Basic,College,2011-01-18,Unemployed,F,...,1,Personal Auto,Personal L3,Offer4,Call Center,744,Four-Door Car,Medsize,NaN,January
2,2,LZ68649,Washington,14947,No,Basic,Bachelor,2011-02-10,Employed,M,...,2,Personal Auto,Personal L3,Offer3,Call Center,480,SUV,Medsize,A,February
3,3,XL78013,Oregon,22332,Yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484,Four-Door Car,Medsize,A,January
5,5,HO81102,Nan,4745,NaN,Basic,High School or Below,2011-02-14,Employed,M,...,7,Personal Auto,Personal L3,Offer1,Agent,292,Four-Door Car,Medsize,A,February
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10904,10904,QI63521,Nevada,19228,No,Basic,High School or Below,2011-02-24,Unemployed,M,...,2,Personal Auto,Personal L2,Offer1,Branch,897,Luxury,Medsize,A,February
10906,10906,KX53892,Oregon,5259,No,Basic,College,2011-01-06,Employed,F,...,6,Personal Auto,Personal L3,Offer2,Branch,273,Four-Door Car,Medsize,A,January
10907,10907,TL39050,Arizona,23893,No,Extended,Bachelor,2011-02-06,Employed,F,...,2,Corporate Auto,Corporate L3,Offer1,Web,381,Luxury,Medsize,NaN,February
10908,10908,WA60547,California,11971,No,Premium,College,2011-02-13,Employed,F,...,6,Personal Auto,Personal L1,Offer1,Branch,618,SUV,Medsize,A,February


In [79]:
pivot5_df = df.pivot_table(index='state', columns='month', values='number_of_policies', aggfunc='count')
pivot5_df

month,February,January
state,,
Arizona,861,953
California,1539,1807
Nan,295,294
Nevada,414,525
Oregon,1274,1480
Washington,399,436


6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

In [89]:
# Find the top 3 states by total number of policies sold
top_states = (df['state'].value_counts().index)
top_states = top_states[:3]
top_states

Index(['California', 'Oregon', 'Arizona'], dtype='object', name='state')

In [93]:
df6 = df[df['state'].isin(top_states)]

In [94]:
df6.groupby(['state', 'month']).agg({'number_of_policies':'count'})

number_of_policies
state      month                       
Arizona    February                 861
           January                  953
California February                1539
           January                 1807
Oregon     February                1274
           January                 1480

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [98]:
# your code goes here
pivot7_df = df.pivot_table(index='response', columns='sales_channel', values='customer', aggfunc='count')
pivot7_df

sales_channel,Agent,Branch,Call Center,Web
response,,,,
No,2967,2412,1669,1252
Yes,703,312,209,164
